# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
J. Müller  ->  J. Müller-Horn  |  ['J. Müller']
Arxiv has 49 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.12227
extracting tarball to tmp_2409.12227...

 done.


P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
J. Sauter  ->  J. Sauter  |  ['J. Sauter']


Found 55 bibliographic references in tmp_2409.12227/pap.bbl.
Retrieving document from  https://arxiv.org/e-print/2409.12232


extracting tarball to tmp_2409.12232... done.


/tmp/ipykernel_2699/1211882699.py:51: LatexWarning: 2409.12232 did not run properly
bad escape \i at position 36
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2409.12261
extracting tarball to tmp_2409.12261... done.


P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
T. Henning  ->  T. Henning  |  ['T. Henning']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']


Found 60 bibliographic references in tmp_2409.12261/aa_submission.bbl.
syntax error in line 5: unbalanced braces
Retrieving document from  https://arxiv.org/e-print/2409.12692
extracting tarball to tmp_2409.12692... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.12227-b31b1b.svg)](https://arxiv.org/abs/2409.12227) | **Observations of microlensed images with dual-field interferometry: on-sky demonstration and prospects**  |
|| P. Mroz, et al. -- incl., <mark>P. Garcia</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-09-20*|
|*Comments*| *submitted to AAS Journals*|
|**Abstract**|            Interferometric observations of gravitational microlensing events offer an opportunity for precise, efficient, and direct mass and distance measurements of lensing objects, especially those of isolated neutron stars and black holes. However, such observations were previously possible for only a handful of extremely bright events. The recent development of a dual-field interferometer, GRAVITY Wide, has made it possible to reach out to significantly fainter objects, and increase the pool of microlensing events amenable to interferometric observations by two orders of magnitude. Here, we present the first successful observation of a microlensing event with GRAVITY Wide and the resolution of microlensed images in the event OGLE-2023-BLG-0061/KMT-2023-BLG-0496. We measure the angular Einstein radius of the lens with a sub-percent precision, $\theta_{\rm E} = 1.280 \pm 0.009$ mas. Combined with the microlensing parallax detected from the event light curve, the mass and distance to the lens are found to be $0.472 \pm 0.012 M_{\odot}$ and $1.81 \pm 0.05$ kpc, respectively. We present the procedure for the selection of targets for interferometric observations, and discuss possible systematic effects affecting GRAVITY Wide data. This detection demonstrates the capabilities of the new instrument and it opens up completely new possibilities for the follow-up of microlensing events, and future routine discoveries of isolated neutron stars and black holes.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.12261-b31b1b.svg)](https://arxiv.org/abs/2409.12261) | **Improving constraints on the extended mass distribution in the Galactic Center with stellar orbits**  |
|| G. Collaboration, et al. -- incl., <mark>P. Garcia</mark>, <mark>S. Scheithauer</mark> |
|*Appeared on*| *2024-09-20*|
|*Comments*| *Submitted to A&A on September 17, 2024*|
|**Abstract**|            Studying the orbital motion of stars around Sagittarius A* in the Galactic Center provides a unique opportunity to probe the gravitational potential near the supermassive black hole at the heart of our Galaxy. Interferometric data obtained with the GRAVITY instrument at the Very Large Telescope Interferometer (VLTI) since 2016 has allowed us to achieve unprecedented precision in tracking the orbits of these stars. GRAVITY data have been key to detecting the in-plane, prograde Schwarzschild precession of the orbit of the star S2, as predicted by General Relativity. By combining astrometric and spectroscopic data from multiple stars, including S2, S29, S38, and S55 - for which we have data around their time of pericenter passage with GRAVITY - we can now strengthen the significance of this detection to an approximately $10 \sigma$ confidence level. The prograde precession of S2's orbit provides valuable insights into the potential presence of an extended mass distribution surrounding Sagittarius A*, which could consist of a dynamically relaxed stellar cusp comprised of old stars and stellar remnants, along with a possible dark matter spike. Our analysis, based on two plausible density profiles - a power-law and a Plummer profile - constrains the enclosed mass within the orbit of S2 to be consistent with zero, establishing an upper limit of approximately $1200 \, M_\odot$ with a $1 \sigma$ confidence level. This significantly improves our constraints on the mass distribution in the Galactic Center. Our upper limit is very close to the expected value from numerical simulations for a stellar cusp in the Galactic Center, leaving little room for a significant enhancement of dark matter density near Sagittarius A*.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.12692-b31b1b.svg)](https://arxiv.org/abs/2409.12692) | **The radial modes of stars with suppressed dipole modes**  |
|| Q. Coppée, <mark>J. Müller</mark>, M. Bazot, S. Hekker |
|*Appeared on*| *2024-09-20*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            The Kepler space mission provided high-quality light curves for more than 16 000 red giants. The global stellar oscillations extracted from these light curves carry information about the interior of the stars. Several hundred red giants were found to have low amplitudes in their dipole modes (i.e. they are suppressed dipole-mode stars). A number of hypotheses (involving e.g. a magnetic field, binarity, or resonant mode coupling) have been proposed to explain the suppression of the modes, yet none has been confirmed. We aim to gain insight into the mechanism at play in suppressed dipole-mode stars by investigating the mode properties (linewidths, heights, and amplitudes) of the radial oscillation modes of red giants with suppressed dipole modes. We selected from the literature suppressed dipole-mode stars and compared the radial-mode properties of these stars to the radial-mode properties of stars in two control samples of stars with typical (i.e. non-suppressed) dipole modes. We find that the radial-mode properties of the suppressed dipole-mode stars are consistent with the ones in our control samples, and hence not affected by the suppression mechanism. From this we conclude that (1) the balance between the excitation and damping in radial modes is unaffected by the suppression, and by extrapolation the excitation of the non-radial modes is not affected either; and (2) the damping of the radial modes induced by the suppression mechanism is significantly less than the damping from turbulent convective motion, suggesting that the additional damping originates from the more central non-convective regions of the star, to which the radial modes are least sensitive.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.12232-b31b1b.svg)](https://arxiv.org/abs/2409.12232) | **UNCOVERing the High-Redshift AGN Population Among Extreme UV Line Emitters**  |
|| H. Treiber, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-09-20*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**|            JWST has revealed diverse new populations of high-redshift ($z\sim4-11$) AGN and extreme star-forming galaxies that challenge current models. In this paper, we use rest-frame UV emission-line diagnostics to identify AGN candidates and other exceptional ionizing sources, complementing previous studies predominantly focused on broad-line AGN. In this paper, we use rest-frame UV emission-line diagnostics to identify AGN candidates and other exceptional ionizing sources, complementing previous studies predominantly focused on broad-line AGN. From a parent sample of 205 $\mathrm{z_{spec}}>3$ UNCOVER galaxies with NIRSpec/PRISM follow-up, we identify 12 C IV, He II, and C III] emitters. Leveraging the combined rest-optical and UV coverage of PRISM, we limit the emission-line model space using the sample's [O III]/H$\beta$ distribution, significantly decreasing the overlap between AGN and star-formation models in the UV diagnostics. We then find that the five He II emitters are the strongest AGN candidates, with further support from two [Ne V] detections and one X-ray detection from Chandra. We cannot robustly quantify the AGN fraction in this sample, but we note that close to 20% of $\mathrm{M_{*}>2\times10^{9}\,M_{\odot}}$ parent sample galaxies are AGN candidates. The lower-mass line emitters, which are consistent with both AGN and star-forming photoionization models, have more compact sizes and higher specific star formation rates than the parent sample. Higher-resolution and deeper data on these UV line emitters should provide much stronger constraints on the obscured AGN fraction at $z > 3$.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 36</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.12227/./events_cdf.png', 'tmp_2409.12227/./parallaxes.png', 'tmp_2409.12227/./cp_epoch1.png']
copying  tmp_2409.12227/./events_cdf.png to _build/html/
copying  tmp_2409.12227/./parallaxes.png to _build/html/
copying  tmp_2409.12227/./cp_epoch1.png to _build/html/
exported in  _build/html/2409.12227.md
    + _build/html/tmp_2409.12227/./events_cdf.png
    + _build/html/tmp_2409.12227/./parallaxes.png
    + _build/html/tmp_2409.12227/./cp_epoch1.png
found figures ['tmp_2409.12261/./orbits.png', 'tmp_2409.12261/./s2_simu_plot.png', 'tmp_2409.12261/./4s_enclmass_pow.png', 'tmp_2409.12261/./4s_enclmass_plum.png']
copying  tmp_2409.12261/./orbits.png to _build/html/
copying  tmp_2409.12261/./s2_simu_plot.png to _build/html/
copying  tmp_2409.12261/./4s_enclmass_pow.png to _build/html/
copying  tmp_2409.12261/./4s_enclmass_plum.png to _build/html/
exported in  _build/html/2409.12261.md
    + _build/html/tmp_2409.12261/./orbits.png
    + _build/html/tmp_2409.12261/./s2

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ra}[4]{{#1}^{\rm h}{#2}^{\rm m}{#3}\fs{#4}}$
$\newcommand{\dec}[4]{{#1}\arcdeg{#2}\arcmin{#3}\farcs{#4}}$
$\newcommand{\rashort}[3]{{#1}^{\rm h}{#2}^{\rm m}{#3}^{\rm s}}$
$\newcommand{\decshort}[3]{{#1}\arcdeg{#2}\arcmin{#3}\arcsec}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Observations of microlensed images with dual-field interferometry: on-sky demonstration and prospects

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.12227-b31b1b.svg)](https://arxiv.org/abs/2409.12227)<mark>Appeared on: 2024-09-20</mark> -  _submitted to AAS Journals_

</div>
<div id="authors">

P. Mróz, et al. -- incl., <mark>P. Garcia</mark>, <mark>L. Kreidberg</mark>, <mark>J. Sauter</mark>

</div>
<div id="abstract">

**Abstract:** Interferometric observations of gravitational microlensing events offer an opportunity for precise, efficient, and direct mass and distance measurements of lensing objects, especially those of isolated neutron stars and black holes. However, such observations were previously possible for only a handful of extremely bright events. The recent development of a dual-field interferometer, GRAVITY Wide, has made it possible to reach out to significantly fainter objects, and increase the pool of microlensing events amenable to interferometric observations by two orders of magnitude. Here, we present the first successful observation of a microlensing event with GRAVITY Wide and the resolution of microlensed images in the event OGLE-2023-BLG-0061/KMT-2023-BLG-0496. We measure the angular Einstein radius of the lens with a sub-percent precision, $\thetaE = 1.280 \pm 0.009$ mas. Combined with the microlensing parallax detected from the event light curve, the mass and distance to the lens are found to be $0.472 \pm 0.012 M_{\odot}$ and $1.81 \pm 0.05$ kpc, respectively. We present the procedure for the selection of targets for interferometric observations, and discuss possible systematic effects affecting GRAVITY Wide data. This detection demonstrates the capabilities of the new instrument and it opens up completely new possibilities for the follow-up of microlensing events, and future routine discoveries of isolated neutron stars and black holes.

</div>

<div id="div_fig1">

<img src="tmp_2409.12227/./events_cdf.png" alt="Fig1" width="100%"/>

**Figure 1. -** Cumulative distribution of the expected $K$-band peak magnitudes of events detected by the OGLE EWS in 2023 (blue line). For comparison, the red line shows the cumulative distribution of the $K$-band magnitudes of the brightest star within $30"$ of the event (fringe-tracking star). Only events with $\tE \geq 50$ day and an adaptive-optics guide star brighter than $G=14$ are plotted. (*fig:stats*)

</div>
<div id="div_fig2">

<img src="tmp_2409.12227/./parallaxes.png" alt="Fig3" width="100%"/>

**Figure 3. -** Constraints on the microlensing parallax derived from different photometric data sets (blue -- OGLE, red -- KMTC, orange -- KMTS, green -- KMTA) and all light curve data (black contours). The dotted and dashed lines mark constraints from the two epochs of VLTI data, respectively. (*fig:parallaxes*)

</div>
<div id="div_fig3">

<img src="tmp_2409.12227/./cp_epoch1.png" alt="Fig6" width="100%"/>

**Figure 6. -** Closure phase data for the VLTI epoch 1. Color solid lines mark the best-fit binary-star model. (*fig:cp1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.12227"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Improving constraints on the extended mass distribution in the Galactic Center with stellar orbits

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.12261-b31b1b.svg)](https://arxiv.org/abs/2409.12261)<mark>Appeared on: 2024-09-20</mark> -  _Submitted to A&A on September 17, 2024_

</div>
<div id="authors">

T. G. Collaboration, et al. -- incl., <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>S. Scheithauer</mark>

</div>
<div id="abstract">

**Abstract:** Studying the orbital motion of stars around Sagittarius A* in the Galactic Center provides a unique opportunity to probe the gravitational potential near the supermassive black hole at the heart of our Galaxy. Interferometric data obtained with the GRAVITY instrument at the Very Large Telescope Interferometer (VLTI) since 2016 has allowed us to achieve unprecedented precision in tracking the orbits of these stars. GRAVITY data have been key to detecting the in-plane, prograde Schwarzschild precession of the orbit of the star S2, as predicted by General Relativity. By combining astrometric and spectroscopic data from multiple stars, including S2, S29, S38, and S55 - for which we have data around their time of pericenter passage with GRAVITY - we can now strengthen the significance of this detection to an approximately $10 \sigma$ confidence level. The prograde precession of S2's orbit provides valuable insights into the potential presence of an extended mass distribution surrounding Sagittarius A*, which could consist of a dynamically relaxed stellar cusp comprised of old stars and stellar remnants, along with a possible dark matter spike. Our analysis, based on two plausible density profiles - a power-law and a Plummer profile - constrains the enclosed mass within the orbit of S2 to be consistent with zero, establishing an upper limit of approximately $1200   M_\odot$ with a $1 \sigma$ confidence level. This significantly improves our constraints on the mass distribution in the Galactic Center. Our upper limit is very close to the expected value from numerical simulations for a stellar cusp in the Galactic Center, leaving little room for a significant enhancement of dark matter density near Sagittarius A*.

</div>

<div id="div_fig1">

<img src="tmp_2409.12261/./orbits.png" alt="Fig1" width="100%"/>

**Figure 1. -** Orbits for the set of 11 S-stars that have been used in this paper. Highlighted are the 4 most relevant stars to constrain the gravitational potential around Sgr A*: S2, S29, S38 and S55. (*fig_orb*)

</div>
<div id="div_fig2">

<img src="tmp_2409.12261/./s2_simu_plot.png" alt="Fig2" width="100%"/>

**Figure 2. -** Predicted error on $f_{SP}$ as a function of time, obtained through mock observations of S2 with GRAVITY and ERIS. The blue points show the result keeping only GRAVITY data in the fit, namely excluding the NACO data and removing the reference frame parameters. The green points show the result keeping both NACO and GRAVITY data.
    (*fig_simu*)

</div>
<div id="div_fig3">

<img src="tmp_2409.12261/./4s_enclmass_pow.png" alt="Fig6.1" width="50%"/><img src="tmp_2409.12261/./4s_enclmass_plum.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Upper limit on the enclosed mass within the orbit of S2 for an extended mass distribution around Sgr A*. We test two plausible density profile for the mass distribution: a power-law profile with varying slope (panel a) and a Plummer profile with varying scale radius (panel b). In red we plot the $1 \sigma$ upper limit and in blue the $3 \sigma$ upper limit, derived from a multi-star fit using data from the stars S2, S29, S38 and S55. Independently of the density profile, the enclosed mass within S2’s orbit is consistently compatible with zero. We set a strong upper limit of approximately $1200   M_\odot$ with a $1 \sigma$ confidence level, for reasonable choices of the slope of the power-law ($s<-1.2$) and the scale radius of the Plummer profile ($a \lesssim 8$ mpc).
        ** Power-law profile****Plummer profile** (*fig:extmass_4s*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.12261"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

187  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
